In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
fxs = pd.read_csv("../input/feature_x_sample.tsv", sep="\t", index_col=0)

cxs = pd.read_csv("../output/comparison_x_sample.tsv", sep="\t", index_col=0)

In [ ]:
def get_log_ratio(is_, v):

    v = kraft.array.log(kraft.array.shift_minimum(v, 1))

    return v[is_ == 1].mean() - v[is_ == 0].mean()

In [ ]:
k_ = {
    "v_data_type": "binary",
    "plot_std": S["plot_std"],
}

for c, is_ in cxs.iterrows():

    is_.dropna(inplace=True)

    m = np.unique(is_, return_counts=True)[1].min()

    for f, s in [
        # [get_log_ratio, 1],
        [kraft.information.get_ic, 2],
    ]:

        if m < s:

            continue

        d = "../output/compare_feature/{}/{}/".format(kraft.path.clean(c), f.__name__)

        kraft.path.make(d)

        ft = kraft.function_heat_map.make(
            is_,
            fxs,
            f,
            n_job=S["n_job"],
            n_sample=S["n_sample"],
            n_shuffle=S["n_shuffle"],
            n_plot=S["n_extreme"],
            title="All ({})".format(f.__name__),
            directory_path=d,
            **k_,
        )

        if 0 < len(S["peek_feature_"]):

            kraft.function_heat_map.make(
                is_,
                fxs.reindex(S["peek_feature_"]),
                ft,
                n_plot=None,
                title="Peek ({})".format(f.__name__),
                **k_,
            )

            plot_peek(ft, S["peek_feature_"], d)